In [22]:
# Import libraries
import sqlite3
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
#print(db_queries.get_ref_id('Mario Yamasaki'))

# connect to database
import os
# get filepath to database
parent_dir = os.path.dirname(os.getcwd())
db_filepath = parent_dir + r'\ufc_db.db'
# connect to database (not needed for now)
db_connection = sqlite3.connect(db_filepath)
cursor = db_connection.cursor()


In [23]:
query = "SELECT fighter_id FROM events JOIN fights ON events.id = fights.event_id JOIN fight_stats ON fights.id = fight_stats.fight_id WHERE fighter_id != ? AND fight_id = ? AND round = 0;"
param = [7, 16]
opp_id = cursor.execute(query, param).fetchone()
print(opp_id)

(31,)


In [45]:
def get_all_date_and_fight_id_for_a_fighter(fighter_id):
  # input: fighter id
  # output: a list of fights (fight_date, fight_id) for 1 fighter
  query = "SELECT events.date, fights.id FROM events JOIN fights ON events.id = fights.event_id WHERE fights.id IN (SELECT fight_id FROM fight_stats WHERE fighter_id = ? AND round = 0);"
  param = [fighter_id]
  return cursor.execute(query, param).fetchall()


def all_fights_of_fighter_before_date(fighter_id, date):
  # input: fighter id + fight date
  # output: a list of fight ids before a given date (of a fight) / None
  
  # format time to datetime obj
  f_date = datetime.strptime(date, '%d/%b/%Y')
  
  return_list = []
  
  # get all of fighter's fights. Loop through. If fight is before input date, add it to return list
  for fight in get_all_date_and_fight_id_for_a_fighter(fighter_id):
    if (datetime.strptime(fight[0], '%d/%b/%Y') < f_date):
      return_list.append(fight[1])

  if (len(return_list) == 0):
    return_list = None

  return return_list

def get_last_n_fights_before_date(fighter_id, date, last_n_fights):
  # This function returns the most recent n amount of fights (id) of a fighter before a date, in chronological order.
  # If fighter does not have that amount of fights, return None
  # input: num of most recent fights to return
  # output: a list of fight id / None
  
  # get all fights of fighter
  #all_fights = all_fights_of_fighter_before_date(fighter_id, date)
  all_fights = [1, 3, 6, 42]
  
  if (all_fights == None) or (last_n_fights < 1):
    return None
  elif (len(all_fights) < last_n_fights):
    return None
  elif (len(all_fights) == last_n_fights):
    return all_fights
  else:
    return_list = []
    counter = 0
    index = len(all_fights)-last_n_fights
    while counter != last_n_fights:
      return_list.append(all_fights[index])
      counter +=1
      index += 1
  return return_list

In [49]:
print(get_last_n_fights_before_date(19, '10/May/2002', 4))

[1, 3, 6, 42]
